In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def preprocess_image(image):
    # resizing and grayscaling the image
    resized_image = cv2.resize(image, (250, 250))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    return(grayscale_image)

# img = cv2.imread("Images/pothole1.jpg")
# processed_image = preprocess_image(img)
# plt.imshow(processed_image, cmap='gray')
# # print(preprocess_image(img))

In [17]:
def measure_depth(image):
    # measure depth (assuming darker regions indicate deeper areas)
    depth_score = np.mean(image)/255.0

    return(depth_score)

In [18]:
def measure_size(image):
    # measuring size of pothole, assuming larger areas indicate larger ppotholes
    size_score = np.count_nonzero(image) / (image.shape[0] * image.shape[1])

    return (size_score)

In [19]:
def measure_texture(image):
    # Assuming more edges indicate rougher texture
    edges = cv2.Canny(image, 50, 150)
    texture_score = np.count_nonzero(edges) / (image.shape[1] * image.shape[1])

    return (texture_score)

In [20]:
def combine_features(depth_score, size_score, texture_score):
    composite_metric = (depth_score + texture_score + size_score) / 3.0

    return composite_metric

In [21]:
def calculate_severity_metric(image):
    image = cv2.imread(image)
    preprocessed_image = preprocess_image(image)
    depth_score = measure_depth(preprocessed_image)
    size_score = measure_size(preprocessed_image)
    texture_score = measure_texture(preprocessed_image)

    composite_metric = combine_features(depth_score, size_score, texture_score)

    return(composite_metric)

In [24]:
image_path = "Images/pothole2.jpg"
severity_metric = calculate_severity_metric(image_path)
print(severity_metric)

0.5695821803921569


In [5]:
# Install the required libraries
# pip install numpy opencv-python mrcnn

import cv2
import numpy as np
from tensorflow import keras
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Define the configuration for the Mask R-CNN model
class PotholeConfig(Config):
    NAME = "pothole"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 2  # Background + Pothole

# Create the Mask R-CNN model
model = modellib.MaskRCNN(mode="inference", config=PotholeConfig(), model_dir='.')

# Load the pre-trained weights for the COCO dataset
model.load_weights('mask_rcnn_coco.h5', by_name=True)

def calculate_pothole_severity(image_path):
    # Read the pothole image
    pothole_image = cv2.imread(image_path)

    # Run the Mask R-CNN model to perform instance segmentation
    results = model.detect([pothole_image], verbose=0)
    r = results[0]

    # Calculate severity based on the area of detected potholes
    severity = 0
    for i in range(r['rois'].shape[0]):
        mask = r['masks'][:, :, i]
        pothole_area = np.sum(mask)
        severity += pothole_area  # You can adjust this metric as needed

    return severity

# Example usage:
image_path = "/Images/pothole1.jpg"
severity_metric = calculate_pothole_severity(image_path)
print("Pothole Severity:", severity_metric)


ModuleNotFoundError: No module named 'keras.engine'